<a href="https://colab.research.google.com/github/sunandhini96/school_of_AI-Assignment10/blob/main/sunandini_assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sunandhini96/Main_Models.git

Cloning into 'Main_Models'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 274 (delta 85), reused 135 (delta 73), pack-reused 108
Receiving objects: 100% (274/274), 82.92 KiB | 10.37 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [2]:
cd /content/Main_Models

/content/Main_Models


In [3]:
from Models import *

In [4]:
!pip install torchinfo
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 1.2 MB/s eta 0:00:00


In [5]:
from Models.vit import *

In [6]:
DATA_DIR='./data'

In [7]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device:", DEVICE)

device: cuda


In [8]:
NUM_CLASSES, IMAGE_SIZE = 10, 32
patch_size=2
mlp_dim=32
dim=32
depth=4
heads=4

model = ViT(IMAGE_SIZE,patch_size,NUM_CLASSES,dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 8, dropout = 0., emb_dropout = 0.)


In [9]:
model.to(DEVICE)

ViT(
  (to_patch_embedding): Sequential(
    (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GELU(approximate='none')
    (2): Conv2d(12, 32, kernel_size=(2, 2), stride=(2, 2))
    (3): Flatten(start_dim=2, end_dim=3)
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (attend): Softmax(dim=-1)
            (to_keys): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (to_queries): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (to_values): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (unifyheads): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (to_out): Sequential(
              (0): Linear(in_features=32, out_features=32, bias=True)
              (1): Dropout(p=0.0, inplace=False)
            )


In [10]:
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters())))

Number of parameters: 48,810


In [11]:
IMAGE_SIZE = 32

NUM_CLASSES = 10
NUM_WORKERS = 8
BATCH_SIZE = 128
EPOCHS = 25

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-1

cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2471, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.75, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandAugment(num_ops=1, magnitude=8),
    transforms.ColorJitter(0.1, 0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
    transforms.RandomErasing(p=0.25)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=4)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [13]:
import time

clip_norm = True
lr_schedule = lambda t: np.interp([t], [0, EPOCHS*2//5, EPOCHS*4//5, EPOCHS], 
                                  [0, 0.01, 0.01/20.0, 0])[0]

model = nn.DataParallel(model, device_ids=[0]).cuda()
opt = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

for epoch in range(EPOCHS):
    start = time.time()
    train_loss, train_acc, n = 0, 0, 0
    for i, (X, y) in enumerate(trainloader):
        model.train()
        X, y = X.cuda(), y.cuda()

        lr = lr_schedule(epoch + (i + 1)/len(trainloader))
        opt.param_groups[0].update(lr=lr)

        opt.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(X)
            loss = criterion(output, y)

        scaler.scale(loss).backward()
        if clip_norm:
            scaler.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(opt)
        scaler.update()
        
        train_loss += loss.item() * y.size(0)
        train_acc += (output.max(1)[1] == y).sum().item()
        n += y.size(0)
        
    model.eval()
    test_acc, m = 0, 0
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X, y = X.cuda(), y.cuda()
            with torch.cuda.amp.autocast():
                output = model(X)
            test_acc += (output.max(1)[1] == y).sum().item()
            m += y.size(0)

    print(f'ViT: Epoch: {epoch} | Train Acc: {train_acc/n:.4f}, Test Acc: {test_acc/m:.4f}, Time: {time.time() - start:.1f}, lr: {lr:.6f}')


ViT: Epoch: 0 | Train Acc: 0.0983, Test Acc: 0.1044, Time: 79.4, lr: 0.001000
ViT: Epoch: 1 | Train Acc: 0.1016, Test Acc: 0.0998, Time: 82.6, lr: 0.002000
ViT: Epoch: 2 | Train Acc: 0.0992, Test Acc: 0.0991, Time: 81.6, lr: 0.003000
ViT: Epoch: 3 | Train Acc: 0.0985, Test Acc: 0.1006, Time: 82.2, lr: 0.004000
ViT: Epoch: 4 | Train Acc: 0.0977, Test Acc: 0.0988, Time: 81.7, lr: 0.005000
ViT: Epoch: 5 | Train Acc: 0.0999, Test Acc: 0.1001, Time: 82.6, lr: 0.006000
ViT: Epoch: 6 | Train Acc: 0.0994, Test Acc: 0.1008, Time: 81.0, lr: 0.007000
ViT: Epoch: 7 | Train Acc: 0.0981, Test Acc: 0.1000, Time: 81.9, lr: 0.008000
ViT: Epoch: 8 | Train Acc: 0.1004, Test Acc: 0.0996, Time: 82.7, lr: 0.009000
ViT: Epoch: 9 | Train Acc: 0.1016, Test Acc: 0.1000, Time: 81.7, lr: 0.010000
ViT: Epoch: 10 | Train Acc: 0.0977, Test Acc: 0.1000, Time: 82.0, lr: 0.009050
ViT: Epoch: 11 | Train Acc: 0.0987, Test Acc: 0.1000, Time: 82.4, lr: 0.008100
ViT: Epoch: 12 | Train Acc: 0.0991, Test Acc: 0.1000, Time: 81